In [ ]:
!git clone https://github.com/UCSD-AI4H/COVID-CT.git

Cloning into 'COVID-CT'...
remote: Enumerating objects: 5459, done.
remote: Total 5459 (delta 0), reused 0 (delta 0), pack-reused 5459
Receiving objects: 100% (5459/5459), 1.09 GiB | 39.56 MiB/s, done.
Resolving deltas: 100% (360/360), done.
Checking out files: 100% (1048/1048), done.


In [ ]:
cd COVID-CT

/content/COVID-CT


In [ ]:
ls


'baseline methods'/       Data-split/         NonCOVID-CT-MetaInfo.csv
 COVID-CT-MetaInfo.xlsx   Images-processed/   README.md


In [ ]:
!unzip Images-processed/CT_COVID.zip >/dev/null; 2>&1
!unzip Images-processed/CT_NonCOVID.zip  > /dev/null; 2>&1

In [ ]:
ls


'baseline methods'/       CT_NonCOVID/        __MACOSX/
 COVID-CT-MetaInfo.xlsx   Data-split/         NonCOVID-CT-MetaInfo.csv
 CT_COVID/                Images-processed/   README.md


In [ ]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten
  
from keras.utils import np_utils

import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

In [ ]:
import cv2
def equalize_light(image, limit=3, grid=(7,7), gray=False):
    if (len(image.shape) == 3):
      clahe = cv2.createCLAHE(clipLimit=limit, tileGridSize=grid)
      lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
      l, a, b = cv2.split(lab)
  
      cl = clahe.apply(l)
      limg = cv2.merge((cl,a,b))
  
      image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
      return np.uint8(image)

In [ ]:
from PIL import Image
import os
import numpy as np
def load_data(folder_path):
  files = os.listdir(folder_path)
  data = []
  count = 0
  for File in files:
    count += 1
    img = Image.open(folder_path+'/'+File)
    img = np.asarray(img,dtype=np.uint8)
    if len(img.shape) > 2:
        # print(img.shape)
        img = Image.fromarray(img)
        img = img.convert('RGB')
        img = np.asarray(img)
        img=cv2.resize(img,(300,300))
        img = equalize_light(img)
        data.append(img)
  return data

In [ ]:
covid,non_covid = load_data('CT_COVID'),load_data('CT_NonCOVID')

In [ ]:
print(len(covid),len(non_covid))

328 374


In [ ]:
sizes  = []
count = 0
x_data = []
y_data = []
left_sum = 0
right_sum = 0
for i in covid:
  x_data.append(i)
  y_data.append(1)
  sizes.append(i.shape)
  left_sum += i.shape[0]
  right_sum += i.shape[1]
  if i.shape[0] >= 299 and i.shape[1] >= 299 :
    count += 1 
for i in non_covid:
  x_data.append(i)
  y_data.append(0)
  left_sum += i.shape[0]
  right_sum += i.shape[1]
  sizes.append(i.shape)
  if i.shape[0] >= 299 and i.shape[1] >= 299 :
    count += 1
print(sizes)
print(len(sizes))
print(len(x_data),len(y_data))
#print(left_sum/702,right_sum/702)

[(300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300, 3), (300, 300

In [ ]:
import sklearn
x_data,y_data = sklearn.utils.shuffle(x_data,y_data)
print(y_data[:5])

x_train,y_train = x_data[:400],y_data[:400]
x_test,y_test = x_data[400:600],y_data[400:600]
x_val,y_val = x_data[600:],y_data[600:]
print(len(x_train),len(y_train))

print(len(x_test),len(y_test))
print(len(x_val),len(y_val))

[0, 0, 1, 1, 1]
400 400
200 200
102 102


In [ ]:
x_train = np.asarray([i for i in x_train])
#x_tr=np.reshape(x_train,())
x_test = np.asarray([i for i in x_test])
x_val = np.asarray([i for i in x_val])

print(x_train.shape)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
y_val = keras.utils.to_categorical(y_val)

(400, 300, 300, 3)


In [ ]:
#print(x_train)
#print(type(x_train))

In [ ]:
def inception_module_a(x,
                      filters_1x1_a,
                      filters_1x1_b,
                      filters_1x1_c,
                      filters_1x1_d,
                      filters_3x3_a,
                      filters_3x3_b,
                      filters_3x3_c,
                      name=None):
    conv_1x1_a = Conv2D(filters_1x1_a, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_1x1_b = Conv2D(filters_1x1_b, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    conv_1x1_c = Conv2D(filters_1x1_c, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3_a = Conv2D(filters_3x3_a, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1x1_a)
    conv_3x3_b = Conv2D(filters_3x3_b, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1x1_b)
    conv_1x1_d = Conv2D(filters_1x1_d, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool)
    
    conv_3x3_c = Conv2D(filters_3x3_c, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3_a)
    
    output = concatenate([conv_1x1_c,conv_1x1_d,conv_3x3_b,conv_3x3_c], axis=3, name=name)
    
    return output

def inception_module_b(x,
                       n,
                      filters_1x1_a,
                      filters_1x1_b,
                      filters_1x1_c,
                      filters_1x1_d,
                      filters_1xn_a,
                      filters_1xn_b,
                      filters_1xn_c,
                      filters_nx1_a,
                      filters_nx1_b,
                      filters_nx1_c,
                      name=None):
    conv_1x1_a = Conv2D(filters_1x1_a, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_1x1_b = Conv2D(filters_1x1_b, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_1x1_c = Conv2D(filters_1x1_c, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    
    conv_1xn_a = Conv2D(filters_1xn_a, (1, n), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1x1_a)
    conv_1xn_b = Conv2D(filters_1xn_b, (1, n), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1x1_b)
    conv_1x1_d = Conv2D(filters_1x1_d, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool)
    
    conv_nx1_a = Conv2D(filters_nx1_a, (n, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1xn_a)
    conv_nx1_b = Conv2D(filters_nx1_b, (n, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1xn_b)
    
    conv_1xn_c = Conv2D(filters_1xn_c, (1,n), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_nx1_a)
    
    conv_nx1_c = Conv2D(filters_1xn_c, (n, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1xn_c)
    
    output = concatenate([conv_1x1_c,conv_1x1_d,conv_nx1_b,conv_nx1_c], axis=3, name=name)
    
    return output

def inception_module_c(x,
                      filters_1x1_a,
                      filters_1x1_b,
                      filters_1x1_c,
                      filters_1x1_d,
                      filters_1x3_a,
                      filters_1x3_b,
                      filters_3x1_a,
                      filters_3x1_b,
                      filters_3x3,
                      name=None):
    conv_1x1_a = Conv2D(filters_1x1_a, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_1x1_b = Conv2D(filters_1x1_b, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_1x1_c = Conv2D(filters_1x1_c, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    
    conv_1x1_d = Conv2D(filters_1x1_d, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool)
    conv_3x3 = Conv2D(filters_3x3, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1x1_a)
    conv_1x3_a = Conv2D(filters_1x3_a, (1, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1x1_b)
    conv_3x1_a = Conv2D(filters_3x1_a, (3, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_1x1_b)
    
    conv_1x3_b = Conv2D(filters_1x3_b, (1, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
    conv_3x1_b = Conv2D(filters_3x1_b, (3,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
    
    output = concatenate([conv_1x1_c,conv_1x1_d,conv_3x1_a,conv_1x3_a,conv_1x3_b,conv_3x1_b], axis=3, name=name)
    
    return output

def inception():
    input_layer = Input(shape=(300,300,3))
    
    x = Conv2D(64, (3,3), strides=(2, 2), activation='relu', name='conv_1_3x3/2_a', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
    x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name='conv_1_3x3/1_a', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    x = Conv2D(64, (3,3), strides=(1,1), padding = 'same', activation='relu', name='conv_1_3x3/1', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name='conv_1_3x3/1_b', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool)
    x = Conv2D(64, (3,3), strides=(2,2), activation='relu', name='conv_1_3x3/2_b', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name='conv_1_3x3/1_c', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    x = inception_module_a(x,16,16,32,64,32,32,32,"a1")
    x = inception_module_a(x,16,16,32,64,32,32,32,"a2")
    x = inception_module_a(x,16,16,32,64,32,32,32,"a3")
    
    x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b1")
    x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b2")
    x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b3")
    x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b4")
    x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b5")
    
    x = inception_module_c(x,32,32,32,64,64,64,32,64,64,"c1")
    x = inception_module_c(x,32,32,32,64,64,64,32,64,64,"c2")
    x = inception_module_c(x,32,32,32,64,64,64,32,64,64,"c3")
    
    if include_top == True:
      pool = MaxPool2D((8,8),strides=(1,1),padding = 'same')(x)

      x = GlobalAveragePooling2D(name='avg_pool3')(pool)

      x = Dropout(0.4)(x)

      x = Dense(2, activation='softmax', name='output3')(x)
    elif include_top == False:
      return x;
    

In [ ]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten

model = Sequential()
input_layer = Input(shape=(300,300,3))
    
x = Conv2D(64, (3,3), strides=(2, 2), activation='relu', name='conv_1_3x3/2_a', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name='conv_1_3x3/1_a', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
x = Conv2D(64, (3,3), strides=(1,1), padding = 'same', activation='relu', name='conv_1_3x3/1', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)

pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name='conv_1_3x3/1_b', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool)
x = Conv2D(64, (3,3), strides=(2,2), activation='relu', name='conv_1_3x3/2_b', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name='conv_1_3x3/1_c', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)

pool1 = MaxPool2D((8,8),strides=(1,1),padding = 'same')(x)

x = inception_module_a(x,16,16,32,64,32,32,32,"a1")
x = inception_module_a(x,16,16,32,64,32,32,32,"a2")
x = inception_module_a(x,16,16,32,64,32,32,32,"a3")

x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b1")
x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b2")
x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b3")
x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b4")
x = inception_module_b(x,7,24,24,24,32,32,16,16,32,32,32,"b5")


pool2 = MaxPool2D((8,8),strides=(1,1),padding = 'same')(x)

x = inception_module_c(x,32,32,32,64,64,64,32,64,64,"c1")
x = inception_module_c(x,32,32,32,64,64,64,32,64,64,"c2")
x = inception_module_c(x,32,32,32,64,64,64,32,64,64,"c3")

pool3 = MaxPool2D((8,8),strides=(1,1),padding = 'same')(x)
InceptionV2 = pool3


In [ ]:
InceptionV2_Model = Model(input_layer,InceptionV2,name='InceptionV2_Model')

In [ ]:
InceptionV2_Model.summary()

Model: "InceptionV2_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv_1_3x3/2_a (Conv2D)         (None, 149, 149, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv_1_3x3/1_a (Conv2D)         (None, 147, 147, 64) 36928       conv_1_3x3/2_a[0][0]             
__________________________________________________________________________________________________
conv_1_3x3/1 (Conv2D)           (None, 147, 147, 64) 36928       conv_1_3x3/1_a[0][0]             
__________________________________________________________________________________

In [ ]:
InceptionV2_Model.load_weights('/content/covid-19_weights1.h5',by_name=True)

In [ ]:
cd ..

/content


In [ ]:
!git clone https://github.com/dongjk/faster_rcnn_keras

Cloning into 'faster_rcnn_keras'...
remote: Enumerating objects: 61, done.
remote: Total 61 (delta 0), reused 0 (delta 0), pack-reused 61
Unpacking objects: 100% (61/61), done.


In [ ]:
!git clone https://github.com/Saitharun279/covid_19

Cloning into 'covid_19'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 14 (delta 0), reused 3 (delta 0), pack-reused 10
Unpacking objects: 100% (14/14), done.


In [ ]:
cd faster_rcnn_keras

/content/faster_rcnn_keras


In [ ]:
!ls

ILSVRC2014_train_00010391.JPEG	RCNN.py  README.md  RPN.py  utils.py


In [ ]:
cd ..

/content


In [ ]:
cd covid_19/data

/content/covid_19/data


In [ ]:
!unzip train.zip
!unzip test.zip

In [ ]:
k = 9

In [ ]:
feature_maps = Input(shape=(None,None,320))

conv_3x3 = Conv2D(
    filters = 128,
    kernel_size = (3,3),
    padding = 'same',
    name = '3x3',
)(feature_maps)

output_rect = Conv2D(
    filters = 4*k,
    kernel_size = (1,1),
    activation = 'linear',
    kernel_initializer='he_uniform',
    name = 'rectangular_object_porposals'
)(conv_3x3)

output_scores = Conv2D(
    filters = 1*k,
    kernel_size = (1,1),
    activation = 'sigmoid',
    kernel_initializer='he_uniform',
    name = 'scores1'
)(conv_3x3)